In [1]:
# !pip install datasets
# !pip install transformers
# !pip install -U accelerate
# !pip install optuna
# !pip install git+https://github.com/yidinghao/interpreting-nlp
# !pip install yattag

#Data

In [2]:
import pickle

with open('train_final.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('test_final.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('val_final.pkl', 'rb') as f:
    X_dev = pickle.load(f)

In [3]:
label_to_class = {
    0: 'none',
    1: 'anger',
    2: 'joy',
    3: 'sadness',
    4: 'love',
    5: 'sympathy',
    6: 'surprise',
    7: 'fear'
}

classes = ['none', 'anger', 'joy', 'sadness', 'love', 'sympathy', 'surprise', 'fear']

In [4]:
X_train.head()

,tweet,label,tweet_with_out_emojis,cleaned_data,Root Stemming,Light Stemming
5081,1996 يا مهند ياحبيبي انت تدور لبش مع اي احد ...,1,1996 يا مهند ياحبيبي انت تدور لبش مع اي احد ...,مهند ياحبيبي تدور لبش متاكد ابو جاسم مايدري,هند حبب تدر لبش تكد ابو جسم ايدر,مهند ياحبيبي تدور لبش متاكد ابو جاسم مايدري
8264,حلب عفرين اعتقال عشرات الشباب من قريه باسوطه ...,3,حلب عفرين اعتقال عشرات الشباب من قريه باسوطه ...,حلب عفرين اعتقال عشرات الشباب قريه باسوطه ريف ...,حلب عفر عقل عشر شبب قره بسط ريف عفر قوت هرب عش...,حلب عفر اعتقال عشر الشباب قريه باسوطه ريف عفر ...
9777,انا نفسي مره اجرب اكون ليك حد اقرب : ),3,انا نفسي مره اجرب اكون ليك حد اقرب : ),مره اجرب اكون ليك اقرب,مره جرب اكن ليك قرب,مره اجرب اكون ليك اقرب
740,الصراحه احساس مخزي ونحن نكتفي بالتمثيل المشرف ...,3,الصراحه احساس مخزي ونحن نكتفي بالتمثيل المشرف ...,الصراحه احساس مخزي نكتفي بالتمثيل المشرف الاول...,صرح حسس خزي كتف شرف اوليمبياد عوز صحه ريض,الصراحه احساس مخزي نكتفي بالتمثيل المشرف الاول...
6537,57 عاجل الايطالي فابيو باسل يفوز بدهب الاولي...,0,57 عاجل الايطالي فابيو باسل يفوز بدهب الاولي...,عاجل الايطالي فابيو باسل يفوز بدهب الاوليمبياد...,عجل يطل ابو بسل يفز دهب اوليمبياد جودو بطل علم...,عاجل الايطالي فابيو باسل يفوز بدهب الاوليمبياد...


In [5]:
X_train = X_train[['tweet', 'Root Stemming', 'label']]
y_train = X_train['label']
X_test = X_test[['tweet', 'Root Stemming', 'label']]
y_test = X_test['label']
X_dev = X_dev[['tweet', 'Root Stemming', 'label']]
y_dev = X_dev['label']

In [6]:
X_train.shape, X_test.shape, X_dev.shape

((7045, 3), (1510, 3), (1510, 3))

In [7]:
X_train

,tweet,Root Stemming,label
5081,1996 يا مهند ياحبيبي انت تدور لبش مع اي احد ...,هند حبب تدر لبش تكد ابو جسم ايدر,1
8264,حلب عفرين اعتقال عشرات الشباب من قريه باسوطه ...,حلب عفر عقل عشر شبب قره بسط ريف عفر قوت هرب عش...,3
9777,انا نفسي مره اجرب اكون ليك حد اقرب : ),مره جرب اكن ليك قرب,3
740,الصراحه احساس مخزي ونحن نكتفي بالتمثيل المشرف ...,صرح حسس خزي كتف شرف اوليمبياد عوز صحه ريض,3
6537,57 عاجل الايطالي فابيو باسل يفوز بدهب الاولي...,عجل يطل ابو بسل يفز دهب اوليمبياد جودو بطل علم...,0
...,...,...,...
5734,احمد الاحمر اوحش لاعب في الاوليمبياد ... اخره ...,حمد حمر وحش لعب اوليمبياد ضخم ناس ولل,3
5191,انتي كان قصدك عليا !,قصد علا,6
5390,يا جدعان لو سمحتوا مش كل واحد عنده ميتين ام عق...,جدع سمح مش وحد ميت ام عقد يرح دخل شتم اسك علج ...,1
860,هتودينا في دهيا يا بلال ارحمنا ده مش وقت قلش,هتد دها بلل رحم مش قلش,1


In [8]:
X_train = X_train.rename_axis('id')
X_test = X_test.rename_axis('id')
X_dev = X_dev.rename_axis('id')

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_dev = X_dev.reset_index(drop=True)

In [9]:
from datasets import Dataset

X_train = Dataset.from_pandas(X_train)
X_test = Dataset.from_pandas(X_test)
X_dev = Dataset.from_pandas(X_dev)
X_train, X_test, X_dev

(Dataset({
     features: ['tweet', 'Root Stemming', 'label'],
     num_rows: 7045
 }),
 Dataset({
     features: ['tweet', 'Root Stemming', 'label'],
     num_rows: 1510
 }),
 Dataset({
     features: ['tweet', 'Root Stemming', 'label'],
     num_rows: 1510
 }))

#Transformer

##Setup

In [10]:
model_checkpoint = 'UBC-NLP/MARBERTv2'

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [12]:
# from transformers import AutoModelForSequenceClassification
from interpret_nlp.modules.lrp_bert_modules import LRPBertForSequenceClassification

# model_init = lambda: AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=8)
model_init = lambda: LRPBertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=8)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["Root Stemming"], padding="max_length", truncation=True, max_length=37)

X_train = X_train.map(tokenize_function, batched=True, num_proc = 8)
X_dev = X_test.map(tokenize_function, batched=True, num_proc = 8)
X_test = X_test.map(tokenize_function, batched=True, num_proc = 8)

Map (num_proc=8):   0%|          | 0/7045 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1510 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1510 [00:00<?, ? examples/s]

In [14]:
X_train

Dataset({
    features: ['tweet', 'Root Stemming', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7045
})

##Training

In [15]:
import numpy as np
from sklearn.metrics import (accuracy_score,
                             f1_score,
                             precision_score,
                             recall_score
                             )

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    assert len(preds) == len(p.label_ids)

    macro_f1 = f1_score(p.label_ids,preds,average='macro', zero_division=1)
    macro_precision = precision_score(p.label_ids,preds,average='macro', zero_division=1)
    macro_recall = recall_score(p.label_ids,preds,average='macro', zero_division=1)
    acc = accuracy_score(p.label_ids,preds)
    return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
    }

In [ ]:
# best run parameters
# dict_items([('learning_rate', 5.9572096694887074e-05), ('num_train_epochs', 4), ('seed', 34), ('per_device_train_batch_size', 16)])

In [16]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy

batch_size = 16
logging_steps = 100
num_epochs = 4

training_args = TrainingArguments(output_dir="./train")
training_args.do_train = True
training_args.evaluate_during_training = True
training_args.per_device_train_batch_size = batch_size
training_args.per_device_eval_batch_size = batch_size
training_args.evaluation_strategy = IntervalStrategy.STEPS
training_args.num_train_epochs = num_epochs
training_args.logging_steps = logging_steps
training_args.eval_steps = logging_steps
training_args.save_steps = 500
training_args.save_total_limit = 5
training_args.seed = 34
training_args.learning_rate = 5.9572096694887074e-05
training_args.load_best_model_at_end=True
training_args.metric_for_best_model = 'macro_f1'

In [17]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_dev,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

Some weights of LRPBertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")
# best_run

In [ ]:
# best_run.hyperparameters.items()
# dict_items([('learning_rate', 1.4788265245160328e-05), ('num_train_epochs', 3), ('seed', 11), ('per_device_train_batch_size', 16)])

In [18]:
# for n, v in best_run.hyperparameters.items():
#     setattr(trainer.args, n, v)

trainer.train()

Some weights of LRPBertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy
100,1.555200,1.270816,0.516018,0.599189,0.541377,0.570861
200,1.200400,1.123235,0.599666,0.629248,0.612721,0.627815
300,1.135400,1.047893,0.629772,0.644144,0.638725,0.646358
400,1.082400,1.083199,0.592961,0.638072,0.611225,0.621854
500,0.935100,1.074629,0.633262,0.673928,0.642376,0.651656
600,0.826600,1.005589,0.660239,0.684478,0.661101,0.670199
700,0.806700,0.922077,0.689001,0.702055,0.690528,0.698013
800,0.778600,0.891301,0.691150,0.703640,0.689393,0.697351


TrainOutput(global_step=800, training_loss=1.04003963470459, metrics={'train_runtime': 165.1561, 'train_samples_per_second': 170.626, 'train_steps_per_second': 10.681, 'total_flos': 243181673349552.0, 'train_loss': 1.04003963470459, 'epoch': 1.81})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8415926694869995,
 'eval_macro_f1': 0.730901437504784,
 'eval_macro_precision': 0.7527250086670499,
 'eval_macro_recall': 0.7317833676274698,
 'eval_accuracy': 0.7397350993377484,
 'eval_runtime': 4.1158,
 'eval_samples_per_second': 366.881,
 'eval_steps_per_second': 23.082,
 'epoch': 1.81}

##Testing

In [19]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
from torch.utils.data import TensorDataset, DataLoader

input_ids = [i['input_ids'] for i in X_test]
attention_masks = [i['attention_mask'] for i in X_test]

inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)
test_data = TensorDataset(inputs, masks)

test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [21]:
from sklearn.metrics import classification_report

model = trainer.model
model.eval()
model.to(device)

test_pred = []
for input, masks in test_dataloader:
    torch.cuda.empty_cache()

    input = input.to(device)
    masks = masks.to(device)

    output = model(input, attention_mask=masks)['logits']
    predictions = torch.argmax(output, dim=1)
    predictions = predictions.cpu().detach().numpy()

    test_pred.extend(predictions)

print(classification_report(X_test['label'], test_pred, target_names=classes))

              precision    recall  f1-score   support

        none       0.64      0.82      0.72       229
       anger       0.48      0.84      0.61       200
         joy       0.54      0.42      0.47       205
     sadness       0.76      0.35      0.48       185
        love       0.80      0.70      0.75       193
    sympathy       0.66      0.79      0.72       156
    surprise       0.56      0.29      0.38       154
        fear       0.96      0.92      0.94       188

    accuracy                           0.65      1510
   macro avg       0.67      0.64      0.63      1510
weighted avg       0.67      0.65      0.64      1510

